# Hypothesis to test:
### Removing objective sentences from reviews helps predict star rating from reviews

In [1]:
import numpy as np
import pandas as pd
import pickle
import gzip
import math
import random
from IPython.display import Markdown, display
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, \
GradientBoostingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, \
classification_report, make_scorer
import statsmodels.api as sm

In [2]:
import sys
sys.path.append('..')

In [3]:
# From this project
from utils import rmse, rmse_train_cv, classifier_report, confusion_rmse
from NLP import WordBag, AboutMovie


# Avoid restarting Kernel
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_colwidth', -1)

# %autosave 50

## Configuration

In [4]:
# Subsampling from Amazon reviews
NB_SAMPLES = 360000 #4000  # up to 200k, then change the input file

data_path = '../../../datasets/'

## Get users' positive and negative reviews

In [5]:
# file_name = '360000_balanced_train_test_reviews.pkl'
file_name = '_balanced_pos_neg_train_test_reviews.pkl'

In [6]:
pickle_in = open(data_path + str(NB_SAMPLES) + file_name,"rb")
train_test_dic0 = pickle.load(pickle_in)

## Subsample

In [7]:
SAMPLE_FRACTION = 0.01

In [8]:
train_test_dic = {'train': {}, 'test':{}}

In [9]:
for i in ['train','test']:
    for j in ['positive','negative']:
         train_test_dic[i][j] = train_test_dic0[i][j] \
            .iloc[:math.floor(len(train_test_dic0[i][j].index) * SAMPLE_FRACTION), :] \
            .drop(['reviewerName', 'helpful', 'summary', 'unixReviewTime', 'reviewTime'], axis=1)
            

## Create Train-CV and Test sets

In [10]:
train_reviews = pd.concat([train_test_dic['train']['positive'],
                     train_test_dic['train']['negative']])
y_train = np.concatenate([np.ones((train_test_dic['train']['positive'].shape[0],)), 
                          np.zeros((train_test_dic['train']['negative'].shape[0],))])
test_reviews = pd.concat([train_test_dic['test']['positive'],
                     train_test_dic['test']['negative']])
y_test = np.concatenate([np.ones((train_test_dic['test']['positive'].shape[0],)), 
                          np.zeros((train_test_dic['test']['negative'].shape[0],))])

In [11]:
train_reviews.shape

(2880, 4)

In [ ]:
train_reviews.head(2)

# => Go down to B testing

In [ ]:
pickle_in = open('../obj_subj_dev/GBC_300_0.5_5.pkl', 'rb')
obj_gbc = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('../obj_subj_dev/fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl', 'rb')
obj_tfidf = pickle.load(pickle_in)
pickle_in.close()


In [ ]:
train_obj_tfidf = obj_tfidf.transform(train_reviews['reviewText']).todense()
                                                 
                                                 

In [ ]:
print(train_obj_tfidf.shape)
train_obj_tfidf[:,10000:10020]

## Create bag of words
Remove accents  
Tokenize  
Lower the case  
Apply custom stop words (keep all negations)  
Remove all non alphabetic characters  
Lematize  
 
Output:  
One list of words for each review 

In [ ]:
train_test_dic['train']['positive'].iloc[:3,:]

In [ ]:
%reload_ext autoreload
word_bag = WordBag()

for i in ['train','test']:
    for j in ['positive','negative']:
        train_test_dic[i][j]['words'] = \
            word_bag.create(train_test_dic[i][j]['reviewText'])

## Remove reviews that may not be on the movie, but on Amazon/support instead
Input: 
* word tokens 
* one line per review 

In [ ]:
%reload_ext autoreload
about_movie = AboutMovie()
movie_reviews = {'train':{}, 'test':{}}
for i in ['train','test']:
    for j in ['positive','negative']:
         movie_reviews[i][j] = train_test_dic[i][j][[about_movie.check(words) \
                                                    for words in train_test_dic[i][j]['words']]]

In [ ]:
# train_test_dic['test']['positive'][[not i for i in \
#                                     [about_movie.check(words) for words in train_test_dic[i][j]['words']]]]

In [ ]:
tot_reviews = 0
for i in ['train','test']:
    for j in ['positive','negative']:
        removed = train_test_dic[i][j].shape[0] - movie_reviews[i][j].shape[0]
        tot_reviews += train_test_dic[i][j].shape[0]
        print('Removed {0} ({1:.0%}) {2} {3} reviews'.format(removed, removed / train_test_dic[i][j].shape[0],
                                                i, j))

In [ ]:
# save results
import datetime
if False:
    currentDT = datetime.datetime.now()
    for i in ['train','test']:
        for j in ['positive','negative']:
            train_test_dic[i][j].to_hdf(data_path + currentDT.strftime("%Y-%m-%d-%H-%M-%S") + '_' + 
                      str(tot_reviews) + '_cleaned_reviews_before_B_' + i + '_' + j + '.pkl'
              , key='df', mode='w', complevel=9)

### TF-IDF setup

In [ ]:
MAX_FEATURES = 10000

In [ ]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',       # Feed a list of words to TF-IDF
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None,
    lowercase=False, 
    stop_words=None, 
    max_features=MAX_FEATURES,
    norm='l2',            # normalize each review
    use_idf=True)        # Keep high weight for most common words

In [ ]:
SPARSE = True

if SPARSE:
    # Optimization: add the review length while keeping sparse matrix
    tf_train = tfidf.fit_transform(train_words)
    tf_test = tfidf.transform(test_words)
else:
    tf_train = tfidf.fit_transform(train_words).todense()
    tf_test = tfidf.transform(test_words).todense()

In [ ]:
# print(len(tfidf.vocabulary_))
# tfidf.vocabulary_

## Add review length to modeling input

In [ ]:
ADD_LENGTH = False

if ADD_LENGTH:
    if SPARSE:
        # Hack: pick an existing word to store the count
        len_idx = 0
        test_lengths = [len(words) for words in test_words]

        for idx,words in enumerate(train_words):
            tf_train[idx][len_idx] = len(words)
        for idx,words in enumerate(test_words):
            tf_test[idx][len_idx] = len(words)
        X_train = tf_train
        X_test = tf_test
    else:
        train_lengths = np.array([len(words) for words in train_words]).reshape(-1,1)
        test_lengths = np.array([len(words) for words in test_words]).reshape(-1,1)
        X_train = np.concatenate([tf_train, train_lengths],axis=1)
        X_test = np.concatenate([tf_test, test_lengths],axis=1)
else:
    X_train = tf_train
    X_test = tf_test

### Test and save

In [ ]:
if X_train.shape[0] != y_train.shape[0] or X_test.shape[0] != y_test.shape[0]:
    print('@@@ Problem! @@@')
    print(X_train.shape)
    print(y_train.shape)
    print(X_test.shape)
    print(y_test.shape)

In [ ]:
if False:
    pickle_out = open(data_path 
                      + 'tfidf_' 
                      + str(X_train.shape[0]) + 'Pos_Neg_Samples_'
                      + str(X_train.shape[1]) + 'Feats.pkl'
                      ,"wb")
    pickle.dump(tfidf, pickle_out)
    pickle_out.close()

## Gradient Boosting Classifier for Base

In [ ]:
# Gradient Boosting Classifier parameters
# N_TREES = math.floor(np.sqrt(NB_SAMPLES) * 1.2)
N_TREES = 500
LEARN_RATE = 0.2
MAX_DEPTH = 8
MIN_IN_LEAF = 5 #7
MAX_FEATURES = 'sqrt'

In [ ]:
gbc = GradientBoostingClassifier(learning_rate=LEARN_RATE, 
                                n_estimators=N_TREES, 
                                min_samples_leaf=MIN_IN_LEAF,
                                max_depth=MAX_DEPTH,
                                max_features=MAX_FEATURES)

In [ ]:
gbc.fit(X_train, y_train)

In [ ]:
if False:
    pickle.dump(gbc, open(data_path + 'GBC_'
                       + str(NB_SAMPLES) + '_samples_'
                       + str(N_TREES) + '_trees_' 
                       + str(LEARN_RATE) + '_lr_' 
                       + str(MAX_DEPTH) + '_maxdpth_'
                       + str(MIN_IN_LEAF) + '_minleaf_'
                       + str(MAX_FEATURES) + '_feats_'
                       + '.pkl', 'wb'))

In [ ]:
%reload_ext autoreload

print(MAX_FEATURES, ' features', N_TREES,'trees; ',
      LEARN_RATE,'learn_rate; ', MAX_DEPTH, 'max_dpth; ',
      MIN_IN_LEAF, 'min_in_leaf')
classifier_report(gbc, X_train, y_train,
                  'Gradient Boosting Classifier on training set')
classifier_report(gbc, X_test, y_test, 
                  'Gradient Boosting Classifier on test set')

In [ ]:
print('SAMPLE_FRACTION:', SAMPLE_FRACTION,'ADD_LENGTH:',ADD_LENGTH,' SPARSE:',SPARSE,' MAX_FEATURES:',MAX_FEATURES)

## Grid search

In [ ]:
if False:
    gb_pipe = Pipeline([('vect', tfidf), ('gb', gbc)])
    gb_pipe.fit(X_train, y_train)
    pickle.dump(gb_pipe, open('pickles/GBCpipe_balanced_comments_'
                           + str(N_TREES) + '_trees_' 
                           + str(LEARN_RATE) + '_lr_' 
                           + str(MAX_DEPTH) + '_maxdpth_'
                           + str(MIN_IN_LEAF) + '_minleaf_'
                           + str(MAX_FEATURES) + '_feats_'
                           + '.pkl', 'wb'))
else:
#     pickle_in = open("pickles/GBC_balanced_comments_300_trees_0.1_lr_15_maxdpth_2_minleaf_20000_feats_.pkl",
#                      "rb")
#     gb_pipe = pickle.load(pickle_in)

In [ ]:
if True:
    grid = {
        'learning_rate': [.1,0.2,0.3],
        'max_depth': [8],
        'min_samples_leaf': [5],
        'max_features': ['sqrt'],
        'n_estimators': [300],
        'random_state': [0]
    }
else:  # TEST
    grid = {
    'learning_rate': [1],
    'max_depth': [2], 
    'min_samples_leaf': [2],
#     'max_features': ['sqrt', None],
    'n_estimators': [2],
    'random_state': [0]
}
    
# confusion_score = make_scorer(confusion_rmse, greater_is_better=False)

gbc_grid_cv = GridSearchCV(
    GradientBoostingClassifier(), 
    grid,
    cv=4,  # number of folds
    return_train_score=True,
    verbose=1, 
    n_jobs=-1)
gbc_grid_cv.fit(X_train, y_train)

In [ ]:
y_pred = gbc_grid_cv.predict(X_test)

print('SAMPLE_FRACTION:', SAMPLE_FRACTION,'ADD_LENGTH:',ADD_LENGTH,' SPARSE:',SPARSE,' MAX_FEATURES:',MAX_FEATURES)

print(gbc_grid_cv.best_params_)
print(gbc_grid_cv.best_score_)
res_df = pd.DataFrame(gbc_grid_cv.cv_results_)
res_df

# Case B: with objective sentences removed

,index,reviewerID,asin,reviewText,overall,sentence
0,1515665,A2D832OA6Q5ZAS,B00005QFFP,"What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known. Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure. I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes. Overall, I am delighted with this video.",4.0,"[What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known., Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure., I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes., Overall, I am delighted with this video.]"
1,1515706,A1JF78EP4GPAOO,B00005QG2N,The concert is fantastic as are the videos and Cradle of Fear trailer. The rest however seems to be uninspired filler. I would have liked this better if it included the Pandaemonaeon music videos and Her Ghost in the Fog. Where was the BBC documentary?,4.0,"[The concert is fantastic as are the videos and Cradle of Fear trailer., The rest however seems to be uninspired filler., I would have liked this better if it included the Pandaemonaeon music videos and Her Ghost in the Fog., Where was the BBC documentary?]"
2,1515714,AQ7K219573Z8P,B00005QIVF,"Lau Ching Wan gives a strong but over-the-top performance as a very rough and mean natured gangster just out of prison, who has been abandoned by his gang. This is kind of a typical story about a very rough guy who has buried very deep a somewhat heart of gold. Both are really taken to extreme in typical Hong Kong style, and as Johnny To often does in many of his films about violent excess in the Hong Kong underworld. So this movie will not appeal to many western tastes. The contrast really is not believable, so it does detract some from the film. But this is made up for by Lau Chin Wan's supurb acting, and by Johnny To's tight directing. This is well worth watching, and is a good, rare Hong Kong drama.",4.0,"[Lau Ching Wan gives a strong but over-the-top performance as a very rough and mean natured gangster just out of prison, who has been abandoned by his gang., This is kind of a typical story about a very rough guy who has buried very deep a somewhat heart of gold., Both are really taken to extreme in typical Hong Kong style, and as Johnny To often does in many of his films about violent excess in the Hong Kong underworld., So this movie will not appeal to many western tastes., The contrast really is not believable, so it does detract some from the film., But this is made up for by Lau Chin Wan's supurb acting, and by Johnny To's tight directing., This is well worth watching, and is a good, rare Hong Kong drama.]"
3,1515719,AINEXVPR5094O,B00005QIVM,storyline: eking cheng is like former gang boss that retired and is now restaurant worker or somthjing...his gf turns into a gangsta re[prsentin...so she tries to bring him back to thegangsta world...pretty ok action...,4.0,[storyline: eking cheng is like former gang boss that retired and is now restaurant worker or somthjing...his gf turns into a gangsta re[prsentin...so she tries to bring him back to thegangsta world...pretty ok action...]


## Split comments into separate sentences

In [12]:
test_reviews.head(1)

,reviewerID,asin,reviewText,overall
1515665,A2D832OA6Q5ZAS,B00005QFFP,"What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known. Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure. I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes. Overall, I am delighted with this video.",4.0


In [15]:
from nltk.tokenize import sent_tokenize
test_reviews['sentence'] = test_reviews['reviewText'].map(sent_tokenize)

In [178]:
test_reviews.head(1)

,reviewerID,asin,reviewText,overall,sentence
1515665,A2D832OA6Q5ZAS,B00005QFFP,"What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known. Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure. I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes. Overall, I am delighted with this video.",4.0,"[What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known., Like a great actress, she dissolves into her role, and yet her voice and style are easily recognizable for the effortless simplicity with which she nails every note and figure., I wish they could have taped in color in those days, and I would have liked to see her perform more examples of the coloratura repertoire and less of the popular romantic themes., Overall, I am delighted with this video.]"


In [57]:
test_reviews.shape

(720, 5)

In [158]:
# WARNING: update test_reviews in 2 places!
sentences = test_reviews['sentence'] \
.apply(pd.Series) \
.merge(test_reviews, left_index = True, right_index = True) \
.drop(['sentence'], axis = 1) \
.melt(id_vars = ['reviewerID', 'asin','overall'], value_name = 'sentence') \
.drop(['variable'], axis = 1) \
.dropna()

print(sentences.shape)

(5647, 4)


In [20]:
sentences.head(10)

,reviewerID,asin,overall,sentence
0,A2D832OA6Q5ZAS,B00005QFFP,4.0,What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known.
1,A1JF78EP4GPAOO,B00005QG2N,4.0,The concert is fantastic as are the videos and Cradle of Fear trailer.
2,AQ7K219573Z8P,B00005QIVF,4.0,"Lau Ching Wan gives a strong but over-the-top performance as a very rough and mean natured gangster just out of prison, who has been abandoned by his gang."
3,AINEXVPR5094O,B00005QIVM,4.0,storyline: eking cheng is like former gang boss that retired and is now restaurant worker or somthjing...his gf turns into a gangsta re[prsentin...so she tries to bring him back to thegangsta world...pretty ok action...
4,AI9AFNGL34D9O,B00005QIVO,4.0,Finally a horror movie that shows something besides girls' cleavage -- guys' cleavage.
5,A3NTHY19HH2D2T,B00005QJHJ,4.0,The choice of some of her best songs is wonderful.
6,A3QADD5SXAQYH2,B00005QJHL,4.0,This is an incredible look at Wings which in my opinion were awesome.There was a 1976 concert called Rockshow which was released on VHS years ago.There is a couple of teasers from that show on Wingspan.Rockshow should be released on DVD!!!!
7,A1Y7DFFCI1XJLX,B00005QJHV,4.0,This is descent low-budget Christmas movie.
8,A8Z468VZVSLL7,B00005QJHV,4.0,Plot was good.
9,A1P918S9GON9NL,B00005QJHV,4.0,"Bought this movie as a gift - haven't opened or viewed it yet - but it arrived in a timely fashion, so I was pleased."


## Vectorize along the word space of the obj-subj training set

In [32]:
obj_model_path = '../obj_subj_dev/'

In [37]:
pickle_in = open(obj_model_path + 'fit_tfidf_vectorizer_for_obj_subj_sentences_classification.pkl', 'rb')
obj_tfidf = pickle.load(pickle_in)
pickle_in.close()
len(obj_tfidf.vocabulary_)

20893

In [38]:
obj_X = obj_tfidf.transform(sentences['sentence']).todense()

In [39]:
obj_X.shape

(5647, 20893)

## Apply the obj-subj model

In [41]:
N_TREES = 100
LEARN_RATE = 0.1
MIN_IN_LEAF = 10
pickle_in = open(obj_model_path + 'GBC_300_0.5_5.pkl', 'rb')
obj_model = pickle.load(pickle_in)

In [43]:
y_test = obj_model.predict(obj_X)
print(len(y_test))
y_test[:10]

5647


array([0, 0, 0, 0, 1, 0, 0, 1, 1, 1])

## Remove objective sentences for case B using obj-subj model

In [159]:
print(sentences.shape)
subjective_sentences = sentences[y_test == 1]

(5647, 4)


In [160]:
diff = len(y_test) - len(subjective_sentences)

display(Markdown('## => Removing {0} ({1:.0%}) objective sentences'
                 .format(diff, diff/len(y_test))))

## => Removing 1766 (31%) objective sentences

### Quality control: objective sentences discarded: really bad filter!!!

In [161]:
sentences.shape

(5647, 4)

In [162]:
sentences[y_test == 0][:10]

,reviewerID,asin,overall,sentence
0,A2D832OA6Q5ZAS,B00005QFFP,4.0,What a pleasure to see this peerless diva perform -- she is the purest example of her art that I have ever known.
1,A1JF78EP4GPAOO,B00005QG2N,4.0,The concert is fantastic as are the videos and Cradle of Fear trailer.
2,AQ7K219573Z8P,B00005QIVF,4.0,"Lau Ching Wan gives a strong but over-the-top performance as a very rough and mean natured gangster just out of prison, who has been abandoned by his gang."
3,AINEXVPR5094O,B00005QIVM,4.0,storyline: eking cheng is like former gang boss that retired and is now restaurant worker or somthjing...his gf turns into a gangsta re[prsentin...so she tries to bring him back to thegangsta world...pretty ok action...
5,A3NTHY19HH2D2T,B00005QJHJ,4.0,The choice of some of her best songs is wonderful.
6,A3QADD5SXAQYH2,B00005QJHL,4.0,This is an incredible look at Wings which in my opinion were awesome.There was a 1976 concert called Rockshow which was released on VHS years ago.There is a couple of teasers from that show on Wingspan.Rockshow should be released on DVD!!!!
10,A2OX72WBLRPZ2,B00005QJHW,4.0,"Arrived super fast & in exellent condition, but i expected to have subtitles in Spanish."
16,A3V2THQU2RQ1AG,B00005QJIF,4.0,"A group of college students (cheerleader, goth, stoner, jock, virgin) decide to visit the old convent that was shut down due to a nasty incident that occurred."
20,A1K4VAYQQLNFVS,B00005QJIG,4.0,"Okay folks, two things: 1."
33,A38BBLT6KV3IW2,B00005QJJL,4.0,Quick synopsis - The Angels have a drug deal is set up but gets foiled when the cops show up.


In [163]:
subjective_sentences.head(2)

,reviewerID,asin,overall,sentence
4,AI9AFNGL34D9O,B00005QIVO,4.0,Finally a horror movie that shows something besides girls' cleavage -- guys' cleavage.
7,A1Y7DFFCI1XJLX,B00005QJHV,4.0,This is descent low-budget Christmas movie.


### Merge the sentences back into paragraph reviews

In [164]:
subj_groups = subjective_sentences.groupby(['reviewerID','asin'])

In [165]:
# can't rely on words, won't be there
def add_space(sentence):
    return ' ' + sentence
def merge_sentences(series):
    s = series.map(add_space)
    res = s.sum()
#     print(res)
#     print('##########')
    return res

subj_reviews_sentences = subj_groups['sentence'].agg(merge_sentences)
print(type(sentences))

<class 'pandas.core.frame.DataFrame'>


In [166]:
subj_reviews_stars = subj_groups['overall'].mean() 

In [167]:
subj_reviews = pd.merge(subj_reviews_sentences, 
                        subj_reviews_stars, 
                        how='inner', on=['reviewerID', 'asin']).reset_index()
subj_reviews.head(2)

,reviewerID,asin,sentence,overall
0,A100RW34WSLTUW,B00005QTA2,"This was predictable, and well done. I went away a little disappointed. It was good, however it could have been written better. This was predictable, and well done. I love Kira, however it was lacking originality. I went away a little disappointed. It was good, however it could have been written better.",4.0
1,A106WW1XZXU3JZ,630369067X,"This particular version of the film is incomplete. The tape ends right in the middle of the movie, literally end of the spindle ends. Unless mine was a fluke copy, which I doubt, the whole movie is not on the tape, which is lame, because you're going to want to see where this film goes. The movie itself is pretty good and weird, but this release of it is the pits. This particular version of the film is incomplete. The tape ends right in the middle of the movie, literally end of the spindle ends. Unless mine was a fluke copy, which I doubt, the whole movie is not on the tape, which is lame, because you're going to want to see where this film goes. The movie itself is pretty good and weird, but this release of it is the pits.",1.0


In [170]:
review_diff = len(test_reviews) - len(subj_review_comments)

display(Markdown('## => Removing {0} ({1:.0%}) reviews with no emotional content'
                 .format(review_diff, review_diff/len(test_reviews))))

## => Removing 23 (3%) reviews with no emotional content

### Check that stars still correspond to the right movie

In [169]:
# test = pd.merge(subj_reviews, )
test_reviews_groups = test_reviews.groupby(['reviewerID','asin'])

test_reviews_stars = test_reviews_groups['overall'].mean()

In [177]:
check = pd.merge(test_reviews_stars, subj_reviews,
                 how='inner', on=['reviewerID', 'asin'])
res = (check['overall_x'] == check['overall_y']).mean()
if res == 1:
    print('OK!')
else:
    print('### @@@@@@@@ PROBLEM! @@@@@@')
    

OK!
